In [1]:
 !wget https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/sentiment-rnn/data/labels.txt
 !wget https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/sentiment-rnn/data/reviews.txt
 

--2020-02-29 13:24:06--  https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/sentiment-rnn/data/labels.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 225000 (220K) [text/plain]
Saving to: ‘labels.txt’

labels.txt          100%[===================>] 219.73K  --.-KB/s    in 0.05s   

2020-02-29 13:24:11 (4.41 MB/s) - ‘labels.txt’ saved [225000/225000]

--2020-02-29 13:24:14--  https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/sentiment-rnn/data/reviews.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200

In [0]:
#In this lesson we will perform Sentiment analysis from Recurret Neural networkimport numpy as np
import numpy as np
with open('reviews.txt', 'r') as f:
    reviews = f.read()
with open('labels.txt', 'r') as f:
    labels = f.read()

In [3]:
print(reviews[:1000])
print(labels[:20])


bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turn

In [0]:
#So it printss the letters and not the lines or words

We ll do Data pre processing

---
1. Getting data into proper form to feed into the network
2. We are using embedded layers 
3. Thus we need to encode each word with an integer
4. Also we need to clean up

---

Here are the processing steps need to be taken
1. Get rid of extraneous punctuation
2. Split the text into each review using \n as the delimiter
3. Combining all reviews back together into one big string





In [0]:
#To remove all the punctuation

from string import punctuation
reviews = reviews.lower() #lowercase
#join concates a string
all_text = ''.join([c for c in reviews if c not in punctuation])

#split by new lines and spaces
reviews_split = all_text.split('\n')
all_text= ''.join(reviews_split)

#create a list of words
words= all_text.split()


In [6]:
words[:30]

['bromwell',
 'high',
 'is',
 'a',
 'cartoon',
 'comedy',
 'it',
 'ran',
 'at',
 'the',
 'same',
 'time',
 'as',
 'some',
 'other',
 'programs',
 'about',
 'school',
 'life',
 'such',
 'as',
 'teachers',
 'my',
 'years',
 'in',
 'the',
 'teaching',
 'profession',
 'lead',
 'me']

In [0]:
#We need to pass integers into the network, by creating a dictionary 
#that maps the words in the vocabulary to integers
#converts each of reviews into integers so they can be passed into network

from collections import Counter

counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vci = {word: ii for ii, word in enumerate(vocab,1)}
reviews_int=[]
for review in reviews_split:
  reviews_int.append([vci[word] for word in review.split()])


In [8]:
print('Words', len(vci))

Words 74072


In [9]:
print('Tokenized review \n', reviews_int[:1])


Tokenized review 
 [[21025, 308, 6, 3, 1050, 207, 8, 2138, 32, 1, 171, 57, 15, 49, 81, 5785, 44, 382, 110, 140, 15, 5194, 60, 154, 9, 1, 4975, 5852, 475, 71, 5, 260, 12, 21025, 308, 13, 1978, 6, 74, 2395, 5, 613, 73, 6, 5194, 1, 24103, 5, 1983, 10166, 1, 5786, 1499, 36, 51, 66, 204, 145, 67, 1199, 5194, 19869, 1, 37442, 4, 1, 221, 883, 31, 2988, 71, 4, 1, 5787, 10, 686, 2, 67, 1499, 54, 10, 216, 1, 383, 9, 62, 3, 1406, 3686, 783, 5, 3483, 180, 1, 382, 10, 1212, 13583, 32, 308, 3, 349, 341, 2913, 10, 143, 127, 5, 7690, 30, 4, 129, 5194, 1406, 2326, 5, 21025, 308, 10, 528, 12, 109, 1448, 4, 60, 543, 102, 12, 21025, 308, 6, 227, 4146, 48, 3, 2211, 12, 8, 215, 23]]


In [0]:
#Encoding
# 1=positive, 0=negative label conversion
labels_split= labels.split('\n')
encoded_labels = np.array([1 if label == 'positive' else 0 for label in labels_split])

As an additional preprocessing step which is removin outliers.
As network expects a standard input text size and so we will shape reviews into specific length

---

1. Getting rid of extremely long ot=r short reviews; the outliers
2. padding/trunctuating the remaining data such that we have reviews of same length

In [12]:
review_lens = Counter([len(x) for x in reviews_int])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 1
Maximum review length: 2514


In [13]:
print('Number of reviews before removing outliers: ', len(reviews_int))

## remove any reviews/labels with zero length from the reviews_ints list.

# get indices of any reviews with length 0
non_zero_idx = [ii for ii, review in enumerate(reviews_int) if len(review) != 0]

# remove 0-length reviews and their labels
reviews_int = [reviews_int[ii] for ii in non_zero_idx]
encoded_labels = np.array([encoded_labels[ii] for ii in non_zero_idx])

print('Number of reviews after removing outliers: ', len(reviews_int))

Number of reviews before removing outliers:  25001
Number of reviews after removing outliers:  25000


PADDING
<br/>
To deal both short and very long reviews, we ll pad our truncate all our reviews to a specific length, we'll pad with 0s.


---
For reviews longer than seq_length, we can truncate them to the first seq_length words. A good seq_length, in this case, is 200.


---
Steps to be followed are:
1. The data should come from review_ints, 
since we want to feed integers to the network.
2. Each row should be seq_length elements long.
3. For reviews shorter than seq_length words, left pad with 0s. That is, if the review is ['best', 'movie', 'ever'], [117, 18, 128] as integers, the row will look like [0, 0, 0, ..., 0, 117, 18, 128].
4. For reviews longer than seq_length, use only the first seq_length words as the feature vector.


---
As a Example
As a small example, if the seq_length=10 and an input review is:

[117, 18, 128]
The resultant, padded sequence should be:

[0, 0, 0, 0, 0, 0, 0, 117, 18, 128]


---

your final features array should be a 2D array, with rows as revies and columns as seq_length





In [0]:
def pad_features(reviews_int, seq_length):
  features = np.zeros((len(reviews_int),seq_length),dtype=int)
  for i, row in enumerate(reviews_int):
    features[i, -len(row):] = np.array(row)[:seq_length]
  return features

In [15]:
seq_length = 200

features = pad_features(reviews_int, seq_length=seq_length)

assert len(features)==len(reviews_int), "Your features should have as many rows as reviews."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 10 values of the first 30 batches 
print(features[:30,:10])

[[    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [22382    42 46418    15   706 17139  3389    47    77    35]
 [ 4505   505    15     3  3342   162  8312  1652     6  4819]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [   54    10    14   116    60   798   552    71   364     5]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    1   330   578    34     3   162   748  2731     9   325]
 [    9    11 10171  5305  1946   689   444    22   280   673]
 [    0     0     0     0     0     0     0     0     0

TRAINING, VALIDATION AND TEST<br />
with our data in nice shape, we'll split into training, validation and test sets
Steps:
1. Create sets for the features and labels, trainx, trainy
2. define split fraction, as the fraction of data to keep in the training set. Usually set to 0.8 or 0.9
3. Whatever data is left will be split in half to create the validation and testing data

In [19]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		(2500, 200)


DataLoaders and Batching <br/>
After creating training, test, and validation data, we can create DataLoaders for this data.
<br >
Steps:<br />
1. Create a known format for accessing our data, using TensorDataset which takes in an input set of data and a target set of data with the same first dimension, and creates a dataset.
2. Create DataLoaders and batch our training, validation, and test Tensor datasets.
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
train_loader = DataLoader(train_data, batch_size=batch_size)


In [20]:
import torch
from torch.utils.data import TensorDataset, DataLoader


train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

batch_size = 50

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 200])
Sample input: 
 tensor([[    0,     0,     0,  ...,    82,    22,  2412],
        [ 5928,  4402,   505,  ...,    31,  1427,   409],
        [    0,     0,     0,  ...,    79,    24,  4337],
        ...,
        [   87,   773,  3992,  ...,     1,   323,   125],
        [   30,   870,   135,  ...,    65,  1581,     8],
        [    7,     7,   712,  ...,   116, 27060,     9]])

Sample label size:  torch.Size([50])
Sample label: 
 tensor([0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
        0, 0])


Sentiment Network with pytorch
<br />
Below is the actual network that you need to define
The layers Steps are: <br />
1. An embeddeing layer that converts our word token into embeddings of a specific size.
2. An LSTM layer defined by a hidden_state size and number of layers
3. A fully convvected output layer that maps the LSTM layer outputs to a desired outputsize
4. asigmoid activation layer which turns all the outputs into a value 0-1; return only the last sigmoid output of this network


---
The embedding layer< br/>
We need to add an embedding layer because there are 74000+ words in our vocabulary.
It is inefficient to one hot encoding instead we can have an embedding layer and use that layer as look up table
we train layer using word2vec


---
The LSTM Layer
we'll create an LSTM to use in our recurrent network, which takes in an inputsize, hiddendim, a number of layers, a droput probability and batchfirst parameter

---
Adding 2-3 layers will let it learn complex relationship





In [0]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

we'll instantiate the network. First up, defining the hyperparameters.

1. vocab_size: Size of our vocabulary or the range of values for our input, word tokens.
2. output_size: Size of our desired output; the number of class scores we want to output (pos/neg).
3. embedding_dim: Number of columns in the embedding lookup table; size of our embeddings.
4. hidden_dim: Number of units in the hidden layers of our LSTM cells. Usually larger is better performance wise. Common values are 128, 256, 512, etc.
5. n_layers: Number of LSTM layers in the network. Typically between 1-3

In [26]:
vocab_size = len(vci)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentRNN(
  (embedding): Embedding(74073, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [0]:

lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [30]:
epochs = 4 # 3-4 is approx where I noticed the validation loss stop decreasing
train_on_gpu=torch.cuda.is_available()
counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/4... Step: 100... Loss: 0.647427... Val Loss: 0.655600
Epoch: 1/4... Step: 200... Loss: 0.545600... Val Loss: 0.634651
Epoch: 1/4... Step: 300... Loss: 0.854420... Val Loss: 0.681114
Epoch: 1/4... Step: 400... Loss: 0.507770... Val Loss: 0.560441
Epoch: 2/4... Step: 500... Loss: 0.518324... Val Loss: 0.525089
Epoch: 2/4... Step: 600... Loss: 0.400961... Val Loss: 0.551876
Epoch: 2/4... Step: 700... Loss: 0.617467... Val Loss: 0.545920
Epoch: 2/4... Step: 800... Loss: 0.390433... Val Loss: 0.481389
Epoch: 3/4... Step: 900... Loss: 0.635266... Val Loss: 0.499887
Epoch: 3/4... Step: 1000... Loss: 0.242123... Val Loss: 0.489482
Epoch: 3/4... Step: 1100... Loss: 0.458671... Val Loss: 0.481863
Epoch: 3/4... Step: 1200... Loss: 0.296621... Val Loss: 0.480806
Epoch: 4/4... Step: 1300... Loss: 0.248170... Val Loss: 0.504206
Epoch: 4/4... Step: 1400... Loss: 0.271310... Val Loss: 0.493889
Epoch: 4/4... Step: 1500... Loss: 0.163220... Val Loss: 0.490412
Epoch: 4/4... Step: 1600... Loss: 

In [31]:
test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output, h = net(inputs, h)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.485
Test accuracy: 0.798
